In [1]:
import os
import sys
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Calculate sales growth

#### Get last three month sales and corresponding months of previous year, and caluclate percentage change. Only need to run at the beginng of month.

In [3]:
from functions import json_get_sales_count, json_get_fooditem_keys, \
    merge_data, modify_into_training_data, get_accesstoken, \
    get_summary_report, get_months_dates

from talech_keys import grant_type, client_id_1, client_id_2, client_secret_1, client_secret_2, \
    client_version, token_url, refresh_token_1, refresh_token_2, ID_HQ, ID_IFSC, ID_PSL, ID_SAS, ID_TCD

In [4]:
import requests
import datetime

In [5]:
accesstoken = get_accesstoken(client_id_1, client_secret_1, refresh_token_1, client_version, token_url)

In [6]:
token = {
    'securityToken': accesstoken,
    'X-POS-MerchantId': ID_SAS
}

In [7]:
targetmonth = 10
targetyear = 2020

In [8]:
prevyear = get_months_dates(-17, 5, targetmonth, targetyear)
prevyearsales = {}
for i in range(1, len(prevyear)):
    salesdata = get_summary_report(token, prevyear[i-1], prevyear[i])
    prevyearsales[prevyear[i-1]] = salesdata

In [9]:
currentyear = get_months_dates(-5, 5, targetmonth, targetyear)
currentyearsales = {}
for i in range(1, len(currentyear)):
    salesdata = get_summary_report(token, currentyear[i-1], currentyear[i])
    currentyearsales[currentyear[i-1]] = salesdata

In [10]:
prevyearsales

{'05/01/2019 00:00:00': {'BAKED GOODS': {'SCONE-HTG': 3,
   'CIN-BUN-TG': 5,
   'CROIS-TG': 4,
   'CARA-BROWN-TG': 3,
   'SCONE-NHSI': 1,
   'CUST-TART-SI': 1,
   'OATCOOK-TG': 2,
   'GRAN-BAR-SI': 1,
   'BBAN-LOAF-SI': 1,
   'CUST-TART-TG': 2,
   'CARR-MUFF-TG': 1,
   'CIN-BUN-SI': 1,
   'ANZAC-TG': 11,
   'BBAN-LOAF-TG': 3,
   'APP-FIN-TG': 4,
   'CROIS-SI': 2,
   'PAIN-RAIS-SI': 1,
   'SCONE-HTSI': 7,
   'PAIN-RAIS-TG': 2,
   'SCONE-NHTG': 10,
   'GRAN-BAR-TG': 2,
   'ANZAC-SI': 1,
   'SNICK-BALL-TG': 3},
  'BREAKFAST': {'HCCROIS-H-TG': 1,
   'TOAST-TG': 1,
   ' POR-CHH-SI': 1,
   'POR-CHH-TG': 3,
   'BERPAR-TG': 2,
   'HCCROIS-NH-TG': 1},
  'LUNCH': {'CHICKBAC-WRAP-NH-TG': 1,
   'TMELT-NH-TG': 1,
   'SOUP-TG': 2,
   'CHICKBAC-WRAP-NH-SI': 2,
   'CHIKSTUF-NH-TG': 2,
   'CHIKSTUF-H-SI': 1}},
 '05/02/2019 00:00:00': {'BAKED GOODS': {'CIN-BUN-TG': 4,
   'PAIN-CHOC-SI': 5,
   'CARA-BROWN-SI': 3,
   'OATCOOK-SI': 3,
   'PAIN-CHOC-TG': 1,
   'CROIS-TG': 1,
   'CARA-BROWN-TG': 3,
   'SCONE

In [11]:
currentyearsales

{'05/01/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/02/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/03/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/04/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/05/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/06/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/07/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/08/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/09/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/10/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/11/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/12/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/13/2020 00:00:00': {'BAKED GOODS': {}, 'BREAKFAST': {}, 'LUNCH': {}},
 '05/14/2020 00:00:00': {

In [12]:
import pandas as pd

In [13]:
prevyearsales = pd.DataFrame(prevyearsales)
currentyearsales = pd.DataFrame(currentyearsales)

In [14]:
def sum_sales_by_category(df):
    salesbycategory = {}
    for column in df.columns:
        salesbycategory[column] = {}
        salesbycategory[column]['dayofweek'] = pd.Timestamp(pd.to_datetime(column)).dayofweek

        for index in df.index:
            item = df.loc[index, column]
            total = sum(item[k] for k in item.keys())
            salesbycategory[column][index] = total
    return salesbycategory

In [15]:
p_salesbycategory = sum_sales_by_category(prevyearsales)
c_salesbycategory = sum_sales_by_category(currentyearsales)

In [16]:
p_salesbycategory

{'05/01/2019 00:00:00': {'dayofweek': 2,
  'BAKED GOODS': 71,
  'BREAKFAST': 9,
  'LUNCH': 9},
 '05/02/2019 00:00:00': {'dayofweek': 3,
  'BAKED GOODS': 103,
  'BREAKFAST': 12,
  'LUNCH': 7},
 '05/03/2019 00:00:00': {'dayofweek': 4,
  'BAKED GOODS': 91,
  'BREAKFAST': 9,
  'LUNCH': 6},
 '05/04/2019 00:00:00': {'dayofweek': 5,
  'BAKED GOODS': 76,
  'BREAKFAST': 4,
  'LUNCH': 1},
 '05/05/2019 00:00:00': {'dayofweek': 6,
  'BAKED GOODS': 47,
  'BREAKFAST': 3,
  'LUNCH': 2},
 '05/06/2019 00:00:00': {'dayofweek': 0,
  'BAKED GOODS': 60,
  'BREAKFAST': 5,
  'LUNCH': 3},
 '05/07/2019 00:00:00': {'dayofweek': 1,
  'BAKED GOODS': 86,
  'BREAKFAST': 17,
  'LUNCH': 10},
 '05/08/2019 00:00:00': {'dayofweek': 2,
  'BAKED GOODS': 84,
  'BREAKFAST': 14,
  'LUNCH': 5},
 '05/09/2019 00:00:00': {'dayofweek': 3,
  'BAKED GOODS': 90,
  'BREAKFAST': 10,
  'LUNCH': 11},
 '05/10/2019 00:00:00': {'dayofweek': 4,
  'BAKED GOODS': 90,
  'BREAKFAST': 13,
  'LUNCH': 7},
 '05/11/2019 00:00:00': {'dayofweek': 5,
 

In [17]:
p_salesbycategory = pd.DataFrame(p_salesbycategory)
c_salesbycategory = pd.DataFrame(c_salesbycategory)

In [18]:
p_salesbycategory

,05/01/2019 00:00:00,05/02/2019 00:00:00,05/03/2019 00:00:00,05/04/2019 00:00:00,05/05/2019 00:00:00,05/06/2019 00:00:00,05/07/2019 00:00:00,05/08/2019 00:00:00,05/09/2019 00:00:00,05/10/2019 00:00:00,...,09/20/2019 00:00:00,09/21/2019 00:00:00,09/22/2019 00:00:00,09/23/2019 00:00:00,09/24/2019 00:00:00,09/25/2019 00:00:00,09/26/2019 00:00:00,09/27/2019 00:00:00,09/28/2019 00:00:00,09/29/2019 00:00:00
dayofweek,2,3,4,5,6,0,1,2,3,4,...,4,5,6,0,1,2,3,4,5,6
BAKED GOODS,71,103,91,76,47,60,86,84,90,90,...,92,62,49,64,67,76,68,88,68,57
BREAKFAST,9,12,9,4,3,5,17,14,10,13,...,14,12,5,21,17,22,25,16,11,7
LUNCH,9,7,6,1,2,3,10,5,11,7,...,18,11,5,15,11,18,12,17,11,2


In [19]:
c_salesbycategory

,05/01/2020 00:00:00,05/02/2020 00:00:00,05/03/2020 00:00:00,05/04/2020 00:00:00,05/05/2020 00:00:00,05/06/2020 00:00:00,05/07/2020 00:00:00,05/08/2020 00:00:00,05/09/2020 00:00:00,05/10/2020 00:00:00,...,09/20/2020 00:00:00,09/21/2020 00:00:00,09/22/2020 00:00:00,09/23/2020 00:00:00,09/24/2020 00:00:00,09/25/2020 00:00:00,09/26/2020 00:00:00,09/27/2020 00:00:00,09/28/2020 00:00:00,09/29/2020 00:00:00
dayofweek,4,5,6,0,1,2,3,4,5,6,...,6,0,1,2,3,4,5,6,0,1
BAKED GOODS,0,0,0,0,0,0,0,0,0,0,...,48,40,60,56,57,66,86,69,41,59
BREAKFAST,0,0,0,0,0,0,0,0,0,0,...,11,16,13,20,19,20,25,16,20,14
LUNCH,0,0,0,0,0,0,0,0,0,0,...,10,14,23,27,20,12,43,35,19,26


In [20]:
def find_average_sales_by_day(df):
    bakedgoods = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    breakfast = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    lunch = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    daysinamonth = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    averages = {}
    
    for date in df.columns:
        if df.loc['BAKED GOODS', date] == 0 and df.loc['BREAKFAST', date] == 0 and df.loc['LUNCH', date] == 0:
            continue
        else:
            week = df.loc['dayofweek', date]
            daysinamonth[str(week)] += 1
            bakedgoods[str(week)] += df.loc['BAKED GOODS', date]
            breakfast[str(week)] += df.loc['BREAKFAST', date]
            lunch[str(week)] += df.loc['LUNCH', date]
                
    totalsalesbycategory = {'bakedgoods':bakedgoods, 'breakfast':breakfast, 'lunch':lunch}
    
    for category, total in totalsalesbycategory.items():
        averages[category] = {}
        for day, sales in total.items():
            averages[category][day] = format(sales/daysinamonth[day], '.1f')

    return averages

In [21]:
p_average = find_average_sales_by_day(p_salesbycategory)
c_average = find_average_sales_by_day(c_salesbycategory)

In [22]:
p_average = pd.DataFrame(p_average)
p_average

,bakedgoods,breakfast,lunch
0,67.4,15.1,10.8
1,72.2,15.4,11.9
2,78.9,16.5,12.5
3,80.9,16.6,12.8
4,88.7,17.8,13.3
5,67.3,7.6,8.7
6,48.4,4.9,4.8


In [23]:
c_average = pd.DataFrame(c_average)
c_average

,bakedgoods,breakfast,lunch
0,37.6,9.1,10.0
1,40.1,8.2,10.5
2,38.8,9.0,10.9
3,40.9,8.1,10.6
4,46.9,9.4,8.8
5,59.3,10.8,15.4
6,41.9,7.9,8.3


In [24]:
# % increase = Increase ÷ Original Number × 100.

In [25]:
from decimal import *

In [26]:
percentagechange = {}
for index in c_average.index:
    percentagechange[index] = {}
    for category in c_average.columns:
        current = c_average.loc[index, category]
        prev = p_average.loc[index, category]
        difference = Decimal(current) - Decimal(prev)
        change = (Decimal(current) - Decimal(prev)) / Decimal(current) * Decimal(100)
        percentagechange[index][category] = format(change, '.1f')

In [27]:
percentagechange = pd.DataFrame(percentagechange)

In [28]:
percentagechange

,0,1,2,3,4,5,6
bakedgoods,-79.3,-80.0,-103.4,-97.8,-89.1,-13.5,-15.5
breakfast,-65.9,-87.8,-83.3,-104.9,-89.4,29.6,38.0
lunch,-8.0,-13.3,-14.7,-20.8,-51.1,43.5,42.2


In [29]:
percentagechange.to_json(r'data/percentile_change/SAS/{}_{}.json'.format(targetmonth, targetyear))

# 